In [1]:
import cv2
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

In [ ]:
RESIZE_W, RESIZE_H = 640, 360
MIN_FRAMES = 20

root = r"C:\Users\lixin\Desktop\E1"

csv_path = os.path.join(root, "AI_features", "merged_metadata.csv")
frame_save_dir = os.path.join(root, "AI_features", "frame_data")

os.makedirs(frame_save_dir, exist_ok=True)

df = pd.read_csv(csv_path)

In [ ]:
def compute_flow(prev, curr):
    flow = cv2.calcOpticalFlowFarneback(
        prev, curr, None,
        0.5, 3, 15, 5, 5, 1.2, 0
    )
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    return mag, ang

def extract_frame_series(video_path, video_id):

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Cannot open video: {video_path}")
        return None

    mags = []
    dir_mean = []
    dir_std = []
    all_dirs = []
    diffs = []

    ret, prev = cap.read()
    if not ret:
        return None

    prev = cv2.resize(prev, (RESIZE_W, RESIZE_H))
    prev_gray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.resize(frame, (RESIZE_W, RESIZE_H))
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        mag, ang = compute_flow(prev_gray, gray)

        mags.append(np.mean(mag))
        dir_mean.append(np.mean(ang))
        dir_std.append(np.std(ang))

        all_dirs.extend(ang.flatten())

        diffs.append(np.mean(cv2.absdiff(prev_gray, gray)))

        prev_gray = gray

    cap.release()

    if len(mags) < MIN_FRAMES:
        return None

    if len(all_dirs) > 0:
        dir_hist, _ = np.histogram(all_dirs, bins=36, range=(0, 2*np.pi))
    else:
        dir_hist = np.zeros(36)

    np.save(os.path.join(frame_save_dir, f"{video_id}_mags.npy"), np.array(mags))
    np.save(os.path.join(frame_save_dir, f"{video_id}_dir_mean.npy"), np.array(dir_mean))
    np.save(os.path.join(frame_save_dir, f"{video_id}_dir_std.npy"), np.array(dir_std))
    np.save(os.path.join(frame_save_dir, f"{video_id}_dir_hist.npy"), dir_hist)
    np.save(os.path.join(frame_save_dir, f"{video_id}_diffs.npy"), np.array(diffs))

    return True

for _, r in tqdm(df.iterrows(), total=len(df)):

    video_id = r["video_id"]
    video_path = r["filepath"]

    extract_frame_series(video_path, video_id)

100%|██████████| 116/116 [2:59:29<00:00, 92.84s/it]    
